In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from matplotlib import pyplot as plt
import seaborn as sns
import vccfcoords

In [23]:
def sim_coordinates(n, ndim, min=0, max=1):

    val_range = max - min
    generated_coordinates = np.random.rand(n, ndim) * val_range + min
    #coord_dict = {x:generated_coordinates[x] for x in list(range(n))}
    
    return generated_coordinates

def get_euclid_dist(pt1, pt2):

    return distance.euclidean(pt1, pt2)

def get_closest_points(coords, target, n):

    distances = distance.cdist([target], coords, metric='euclidean')[0]

    idx = np.argsort(distances)[:n]

    return idx
    
def dict_to_array(coord_dict):

    return np.vstack(list(coord_dict.values()))


def plot_coordinates(coords, ndim):

    assert(ndim in [2,3])

    #coords = dict_to_array(coord_dict)

    if ndim == 2:
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]

        plt.scatter(x_vals, y_vals)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()

        return
    
    else:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Generate the values
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]
        z_vals = coords[:, 2:3]

        # Plot the values
        ax.scatter(x_vals, y_vals, z_vals, c = 'b', marker='o')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')

        plt.show()

        return


In [54]:
#Generating Cell Coordinates
EC_coords = sim_coordinates(1000, 3, -1, 1)
other_cell_coords = sim_coordinates(10000, 3, -1, 1)

#Computing Closest Neighbors to Each Cell
closest_EC_cells_3 = {x:get_closest_points(EC_coords,other_cell_coords[x], 4) for x in list(range(len(other_cell_coords)))}

In [55]:
closest_EC_cells_3

{0: array([567, 200, 158, 926]),
 1: array([555, 842, 777, 188]),
 2: array([769, 967, 280, 762]),
 3: array([356, 354,  11, 218]),
 4: array([708, 341, 913, 954]),
 5: array([864, 526, 963,  17]),
 6: array([534, 693, 962, 484]),
 7: array([691, 234, 133, 821]),
 8: array([625, 257, 396, 564]),
 9: array([991, 864, 592, 526]),
 10: array([229, 637, 630, 293]),
 11: array([449, 603, 237, 811]),
 12: array([234, 133, 746, 692]),
 13: array([669,  72, 524, 204]),
 14: array([202, 164, 947, 892]),
 15: array([ 17, 963, 446, 432]),
 16: array([941,   2, 125, 216]),
 17: array([782, 252, 684, 359]),
 18: array([174, 209, 271, 618]),
 19: array([607, 221, 348, 551]),
 20: array([991, 864,   2, 526]),
 21: array([ 27,  96, 604, 261]),
 22: array([600, 839, 235,  69]),
 23: array([887, 266, 736, 246]),
 24: array([811, 377, 610, 237]),
 25: array([880, 827, 521, 650]),
 26: array([114, 916, 676, 966]),
 27: array([277, 328, 895, 997]),
 28: array([526, 381, 442, 955]),
 29: array([899, 499, 13

In [56]:
other_cell_coords[0]

array([-0.51342306,  0.68323159, -0.92591545])

In [57]:
EC_coords[closest_EC_cells_3[0]]

array([[-0.40882984,  0.67813756, -0.94476455],
       [-0.62338338,  0.6155872 , -0.92901035],
       [-0.50300839,  0.75957002, -0.81990308],
       [-0.63502526,  0.71578079, -0.87368287]])